In [1]:
# Mount into drive
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd 'drive/MyDrive/DL7643_Group_project/'
# Verify the contents of the current folder
!ls 

/content/drive/MyDrive/DL7643_Group_project
 CSV_Log			        no_text_100.zip
 deepfill2_3.28.zip		        no_text_30
'deepfill2_3.28.zip (Unzipped Files)'   no_text_30.zip
 defaults.yaml			        no_text_50
 Generate_data.ipynb		        no_text_50.zip
 hateful_memes.zip		        no_text_70
'hateful_memes.zip (Unzipped Files)'    no_text_70.zip
'list of models.gdoc'		       'Project Plan.gdoc'
 memes900k.zip			        remove_text
 memotion_dataset_7k.zip	        Results.gsheet
 no_text_100			        vilbert


In [3]:
!pip install jsonlines
import pandas as pd
import jsonlines
import shutil
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [4]:
pwd

'/content/drive/MyDrive/DL7643_Group_project'

In [5]:
# t = "test"
t = "val"

# c = "soft"
# c = "weighted"
c = "hard"

In [6]:
if t == "val":
  result1 = pd.read_csv('CSV_Log/vilbert_13_val.csv')  
  result2 = pd.read_csv('CSV_Log/vilbert_cc_1_val.csv')  
  result3 = pd.read_csv('CSV_Log/visual_bert_2_val.csv')  
  result4 = pd.read_csv('CSV_Log/visual_bert_coco_3_val.csv')  
  # result5 = pd.read_csv('CSV_Log/mmbt_1_val.csv')  

else:
  result1 = pd.read_csv('CSV_Log/vilbert_13_test.csv')  
  result2 = pd.read_csv('CSV_Log/vilbert_cc_1_test.csv')  
  result3 = pd.read_csv('CSV_Log/visual_bert_2_test.csv')  
  result4 = pd.read_csv('CSV_Log/visual_bert_coco_3_test.csv')  
  # result5 = pd.read_csv('CSV_Log/mmbtt_1_test.csv')  

# result5 = pd.read_csv('CSV_Log/mmbt_1_val.csv')  
result1.columns = ["id","prob_vilbert","label_vilbert"]
result2.columns = ["id","prob_vilbert_cc","label_vilbert_cc"]
result3.columns = ["id","prob_visual_bert","label_visual_bert"]
result4.columns = ["id","prob_visual_bert_coco","label_visual_bert_coco"]
# result5.columns = ["id","prob_mmbt","label_mmbt"]

In [7]:
result1.head()

,id,prob_vilbert,label_vilbert
0,71680,0.999987,1
1,13750,0.000047,0
2,83045,0.998671,1
3,97305,0.999786,1
4,86471,0.000015,0


In [8]:
result = result1.merge(result2, on='id', how='left')
result = result.merge(result3, on='id', how='left')
result = result.merge(result4, on='id', how='left')
# result = result.merge(result5, on='id', how='left')

In [9]:
result.dtypes

id                          int64
prob_vilbert              float64
label_vilbert               int64
prob_vilbert_cc           float64
label_vilbert_cc            int64
prob_visual_bert          float64
label_visual_bert           int64
prob_visual_bert_coco     float64
label_visual_bert_coco      int64
dtype: object

In [10]:
result.head()

,id,prob_vilbert,label_vilbert,prob_vilbert_cc,label_vilbert_cc,prob_visual_bert,label_visual_bert,prob_visual_bert_coco,label_visual_bert_coco
0,71680,0.999987,1,9.999971e-01,1,0.641323,1,0.993914,1
1,13750,0.000047,0,7.204230e-05,0,0.008074,0,0.000441,0
2,83045,0.998671,1,8.896011e-01,1,0.606743,1,0.020548,0
3,97305,0.999786,1,9.573836e-01,1,0.930001,1,0.988170,1
4,86471,0.000015,0,6.648883e-07,0,0.000557,0,0.000296,0


In [11]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

scores = [0.7167, 0.7135, 0.712, 0.7246,0.665]
print(softmax(scores))

[0.20203403 0.20138856 0.2010867  0.20363642 0.19185429]


In [12]:
result['soft_avg'] = result[['prob_vilbert', 'prob_vilbert_cc','prob_visual_bert','prob_visual_bert_coco']].mean(axis=1)
weight_1 = scores[0]
weight_2 = scores[1]
weight_3 = scores[2] 
weight_4 = scores[3]

weight_1 = 0.3
weight_2 = 0.2
weight_3 = 0.1
weight_4 = 0.4
# weight_5 = scores[4]

result['soft_weighted'] = result['prob_vilbert'] * weight_1 + result['prob_vilbert_cc'] * weight_2 + result['prob_visual_bert'] * weight_3 + result['prob_visual_bert_coco'] * weight_4 
# +result['prob_mmbt'] * weight_5

result['hard_vote'] = result[['label_vilbert', 'label_vilbert_cc','label_visual_bert','label_visual_bert_coco']].sum(axis=1)

In [13]:
!ls "hateful_memes.zip (Unzipped Files)/hateful_memes"

dev_seen.jsonl	  img	       README.md	test_unseen.jsonl
dev_unseen.jsonl  LICENSE.txt  test_seen.jsonl	train.jsonl


In [14]:

id = []
true_label = []
if t == "val":
  with jsonlines.open('hateful_memes.zip (Unzipped Files)/hateful_memes/dev_unseen.jsonl') as f:
      for line in f.iter():
        id.append(int(line["id"]))
        true_label.append(line["label"])
else:
  with jsonlines.open('hateful_memes.zip (Unzipped Files)/hateful_memes/test_unseen.jsonl') as f:
      for line in f.iter():
        id.append(int(line["id"]))
        true_label.append(line["label"])


In [15]:
true_label_df = pd.DataFrame(list(zip(id, true_label)), columns =['id', 'true_label']) 

In [16]:
merge_df = result.merge(true_label_df, on='id', how='left')
merge_df["agg_label"] = 0
if c == "soft":
  merge_df.loc[(merge_df.soft_avg >=0.5), 'agg_label'] = 1
if c == "weighted":
  merge_df.loc[(merge_df.soft_weighted >=0.5), 'agg_label'] = 1
if c == "hard":
  merge_df.loc[(merge_df.hard_vote > 2), 'agg_label'] = 1


In [17]:
merge_df.head(100)

,id,prob_vilbert,label_vilbert,prob_vilbert_cc,label_vilbert_cc,prob_visual_bert,label_visual_bert,prob_visual_bert_coco,label_visual_bert_coco,soft_avg,soft_weighted,hard_vote,true_label,agg_label
0,71680,0.999987,1,9.999971e-01,1,0.641323,1,0.993914,1,0.908805,0.961693,4,1,1
1,13750,0.000047,0,7.204230e-05,0,0.008074,0,0.000441,0,0.002158,0.001012,0,0,0
2,83045,0.998671,1,8.896011e-01,1,0.606743,1,0.020548,0,0.628891,0.546415,3,1,1
3,97305,0.999786,1,9.573836e-01,1,0.930001,1,0.988170,1,0.968835,0.979680,4,1,1
4,86471,0.000015,0,6.648883e-07,0,0.000557,0,0.000296,0,0.000217,0.000179,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,31702,0.000152,0,2.317630e-01,0,0.171648,0,0.483129,0,0.221673,0.256814,0,0,0
96,26530,0.000003,0,7.180548e-06,0,0.001420,0,0.000882,0,0.000578,0.000497,0,0,0
97,69015,0.965252,1,7.435935e-01,1,0.103165,0,0.931988,1,0.686000,0.821406,3,0,1
98,3214,0.010063,0,4.138630e-06,0,0.004540,0,0.013959,0,0.007141,0.009057,0,0,0


In [18]:
if c == "soft":
  print("Accuracy",accuracy_score(merge_df.true_label, merge_df.agg_label))
  print("F1",f1_score(merge_df.true_label, merge_df.agg_label))
  print("ROC",roc_auc_score(merge_df.true_label, merge_df.soft_avg))
if c == "weighted":
  print("Accuracy",accuracy_score(merge_df.true_label, merge_df.agg_label))
  print("F1",f1_score(merge_df.true_label, merge_df.agg_label))
  print("ROC",roc_auc_score(merge_df.true_label, merge_df.soft_weighted))

if c == "hard":
  print("Accuracy",accuracy_score(merge_df.true_label, merge_df.agg_label))
  print("F1", f1_score(merge_df.true_label, merge_df.agg_label))
  # print("ROC",roc_auc_score(merge_df.true_label, merge_df.soft_weighted))

# val - soft avg
# Accuracy 0.7148148148148148
# F1 0.5361445783132531
# ROC 0.7463823529411765

# val - weighted
# Accuracy 0.7185185185185186
# F1 0.5421686746987951
# ROC 0.7464411764705882

# test - soft avg
# Accuracy 0.724
# F1 0.5415282392026577
# ROC 0.7789322666666667

# test - weighted
# Accuracy 0.7285
# F1 0.5493775933609959
# ROC 0.7788309333333332

# val - hard
# Accuracy 0.7111111111111111
# F1 0.4834437086092716

# test - hard
# Accuracy 0.7135
# F1 0.4906666666666666

Accuracy 0.7111111111111111
F1 0.4834437086092716


In [20]:
# TO do:
# hard voting: cal accuracy f1
# soft voting: weight
0.7943 - 0.779

0.01529999999999998